# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1.What test is appropriate for this problem? Does CLT apply?
Yes, the CLT applies because there is a large sample size. A z-test for the difference in proportions is appropriate for this problem, because we are testing whether there is a difference in proportions.
Sample size>30 so z-statistic is appropriate

2. What are the null and alternate hypotheses?
Ho: There is no significant difference between "percentage called back" for black and white resumes. 
H1: There is significant difference between "percentage called back" for black and white resumes. 

3. Compute margin of error, confidence interval, and p-value.
Caclulated the confidence interval for the difference in proportions. First, we can compute the proportion of white-sounding and black-sounding candidates who receive calls:

In [24]:
# Separating the dataframe into black and white
b_call = sum(data[data.race=='b'].call)
w_call = sum(data[data.race=='w'].call)
# Checking 'n' of each sample
b_size = data[data.race=='b'].race.size
w_size = data[data.race=='w'].race.size

In [25]:
# Proportion of blacks that were called
b_prop=b_call/b_size
b_prop

0.06447638603696099

In [31]:
# % of back and white callbacks
b_per = b_prop*100
w_per = w_prop*100

In [26]:
# Proportion of white that were called
w_prop=w_call/w_size
w_prop

0.09650924024640657

In [27]:
diff=w_prop-b_prop
diff

0.032032854209445585

# Computing Margin of error

In [28]:
# Finding standard error for proportions 
SE_diff = ((b_prop * (1 - b_prop) / 2435) + (w_prop * (1 - w_prop) / 2435))**0.5

print(SE_diff)

0.0077833705866767544


# Computing Confidence of Interval

In [29]:
# Calculate confidence of interval for the difference in proportions using the formula  1.96* SE, 
# with 1.96 being the z-value corresponding to 95% confidence.
CI_diff = [w_prop - b_prop - 1.96 * SE_diff, w_prop - b_prop + 1.96 * SE_diff]
print(CI_diff)

[0.016777447859559147, 0.047288260559332024]


Using 95% confidence level as threshold in all calculations, we can say that there is 95% confidence that confidence interval of "difference in % callbacks in black and white sounding resume" is from 0.017 to 0.04

In [30]:
# Calculate z-test
from statsmodels.stats.proportion import proportions_ztest

count = np.array([w_call, b_call])
n_obs = np.array([2435, 2435])

z, p = proportions_ztest(count, n_obs)

print("Z statistic: " + str(z))
print("p-value: " + str(p))

Z statistic: 4.108412152434346
p-value: 3.983886837585077e-05


In [ ]:
Since the p_value is less than 0.05, we can reject the null hypothesis and conclude that there is a difference in call rates
between applicants based on whether their names sound white or black.

4. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [ ]:
The above analysis does not show that the perceived race of a candidates name is the most important factor in callback success.
It just shows that race makes a statistically significant difference, not that the race difference is greater than from 
any other factors. 
To determine whether race is the most important, confidence intervals and z-tests need to be calculated for the other columns 
in the data and compare the effect sizes and p-values with the results from this analysis.